In [1]:
import glob
import json
import time
import evaluate
import transformers
import numpy as np
import pandas as pd

from pathlib import Path
from tqdm.auto import tqdm
from datasets import Dataset
from tokenizers import Tokenizer
from tokenizers.normalizers import (Sequence, Lowercase, NFD, StripAccents)
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.decoders import BPEDecoder
from transformers import AutoConfig, \
    DataCollatorWithPadding, AutoModelForSequenceClassification, \
    Trainer, TrainingArguments, AutoTokenizer, GPT2Config
from matplotlib import pyplot as plt
import torch
from transformers import get_scheduler, AdamW

c:\Users\bunna\AppData\Local\Programs\Python\Python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.16) or chardet (5.1.0)/charset_normalizer (2.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
def top_x_acc(y_true, y_pred, x):
    y_true = torch.tensor(y_true)
    y_pred = torch.tensor(y_pred)
    ranked = torch.argsort(y_pred, axis=-1)
    top_x = ranked[..., -x:]
    return (top_x == torch.repeat_interleave(y_true.unsqueeze(-1), x, axis=-1)).float().sum(-1).mean().item()

def mean_recip_rank(y_true, y_pred):
    y_true = torch.tensor(y_true)
    y_pred = torch.tensor(y_pred)
    ranked = torch.argsort(y_pred, axis=-1)
    true_ranks = y_pred.shape[-1] - (ranked == torch.repeat_interleave(y_true.unsqueeze(-1), y_pred.shape[-1], axis=-1)).float().argmax(-1)

    return (1/true_ranks).mean().item()

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # predictions = np.argmax(predictions, axis=1)

    top_one = top_x_acc(labels, predictions, 1)
    top_five = top_x_acc(labels, predictions, 5)
    top_ten = 0
    if predictions.shape[-1] >= 10:
        top_ten = top_x_acc(labels, predictions, 10)
    mrr = mean_recip_rank(labels, predictions)

    metrics = {"top_one" : top_one, "top_five" : top_five, "mrr": mrr, "top_ten": top_ten}
    return metrics

In [3]:
def merge_staff_overlaps(bscores):
    """
    Takes in either one binary score or a batch of them and merges the left and right hands
    """
    
    # Lower middle c is index 23
    # Upper middle c is index 33
    lower = 23
    upper = 33
    middle = (lower + upper) // 2
    
    # Total notes is 52
    total = 52
    
    # Pad out upper hand and lower hand and combine them
    padded_lower = np.concatenate([bscores[..., :middle], np.zeros((*bscores.shape[:-1], total-middle))], axis=-1)
    padded_upper = np.concatenate([np.zeros((*bscores.shape[:-1], middle-bscores.shape[-1]+total)), bscores[..., middle:]], axis=-1)
    # Logical or
    merged = padded_lower + padded_upper - padded_lower * padded_upper
    return merged


# Continuous line of 256 unicode characters
start = 10060# 931
dense_characters = [chr(i).encode("utf-8").decode("utf-8") for i in range(start, start+512)]


# This code divides the fragment into blocks (and discards any remaining info at the very edges)
# Then it uses einsum with a filter of powers of 2 to convert from binary to an integer.  Then converts integers into
# unicode characters

def dense_encoder(fragment, block_size=[1, 1]):
    fragment = merge_staff_overlaps(fragment)
    # Rewrote this to be much faster but looks complicated
    # This filter has powers of 2 which is how the binary is turned to ints
    filter_ = np.power(2, np.arange(np.prod(block_size))).reshape(block_size)
    
    # The fragment is split into blocks here
    xblocks = np.stack(np.split(fragment[:, :(fragment.shape[1]//block_size[1])*block_size[1]], fragment.shape[1]//block_size[1], axis=1))
    xyblocks = np.stack(np.split(xblocks[:, :(xblocks.shape[1]//block_size[0])*block_size[0]], xblocks.shape[1]//block_size[0], axis=1))
    
    # The blocks are multiplied so they are ints
    numbers = np.einsum("ijkl,kl->ij", xyblocks, filter_)
    
    # The ints are turned into corresponding characters
    characters = (numbers+start).astype(np.int32).view('U1')
    return " ".join(["".join(t) for t in characters])

def data_preparation(labeled_data):
    train_X, train_y, val_X, val_y, test_X, test_y, train_m, valid_m, test_m = pd.read_pickle(labeled_data)

    train_df = pd.DataFrame({"text": [dense_encoder(piece, block_size=[1,8]) for piece in train_X], "label": train_y})
    val_df = pd.DataFrame({"text": [dense_encoder(piece, block_size=[1,8]) for piece in val_X], "label": val_y})
    test_df = pd.DataFrame({"text": [dense_encoder(piece, block_size=[1,8]) for piece in test_X], "label": test_y})
    
    return train_df, val_df, test_df

def label2id_function(examples, label2id):
    return {"label": [label2id[label] for label in examples["label"]]}

def tokenizer_function(examples, tokenizer):
    return tokenizer(examples["text"], padding='max_length', truncation=True)

In [4]:
gpt2_dir = Path("./")
seed = 42

tokenizer = AutoTokenizer.from_pretrained(gpt2_dir/"roberta_pretrained")
tokenizer.pad_token = '<pad>'

In [14]:
labeled_data = "100_way_dataset.pkl"
train_df, val_df, test_df = data_preparation(labeled_data)
train_ds = Dataset.from_dict(train_df)
val_ds = Dataset.from_dict(val_df)
test_ds = Dataset.from_dict(test_df)

label2id = {label: i for i, label in enumerate(set(train_df['label']))}
id2label = {i: label for label, i in label2id.items()}

train_ds = train_ds.map(tokenizer_function, batched=True, fn_kwargs={"tokenizer": tokenizer})
train_ds = train_ds.map(label2id_function, batched=True, fn_kwargs={"label2id": label2id})
val_ds = val_ds.map(label2id_function, batched=True, fn_kwargs={"label2id": label2id})
val_ds = val_ds.map(tokenizer_function, batched=True, fn_kwargs={"tokenizer": tokenizer})
test_ds = test_ds.map(tokenizer_function, batched=True, fn_kwargs={"tokenizer": tokenizer})
test_ds = test_ds.map(label2id_function, batched=True, fn_kwargs={"label2id": label2id})

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest", max_length=1024)

Map: 100%|██████████| 15000/15000 [00:00<00:00, 279793.12 examples/s]


In [15]:
config = AutoConfig.from_pretrained(gpt2_dir/"roberta_pretrained")
config.num_labels = len(label2id)
config.pad_token_id = tokenizer.pad_token_id
model = AutoModelForSequenceClassification.from_pretrained(gpt2_dir/"roberta_pretrained", config=config)

# Freeze all layers except the classifier
for name, param in model.named_parameters():
    param.requires_grad = False
model.classifier.out_proj.weight.requires_grad = True

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta_pretrained and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
training_args = TrainingArguments(
    output_dir="C:\\Users\\bunna\\Desktop\\classifier_100",
    learning_rate=5e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=12,
    weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# Train model
trainer.train()

  8%|▊         | 2188/26256 [01:07<12:09, 32.97it/s]

{'loss': 4.5918, 'learning_rate': 4.5833333333333334e-05, 'epoch': 1.0}


                                                    
  8%|▊         | 2188/26256 [01:21<12:09, 32.97it/s]

{'eval_loss': 4.548223972320557, 'eval_top_one': 0.029600000008940697, 'eval_top_five': 0.10406666994094849, 'eval_mrr': 0.08791352063417435, 'eval_top_ten': 0.17746666073799133, 'eval_runtime': 13.2253, 'eval_samples_per_second': 1134.194, 'eval_steps_per_second': 35.462, 'epoch': 1.0}


 17%|█▋        | 4376/26256 [02:28<11:17, 32.30it/s]  

...

                                                     
100%|██████████| 26256/26256 [16:16<00:00, 32.17it/s]

{'eval_loss': 4.375930309295654, 'eval_top_one': 0.05426666513085365, 'eval_top_five': 0.16899999976158142, 'eval_mrr': 0.1298314779996872, 'eval_top_ten': 0.26899999380111694, 'eval_runtime': 13.7363, 'eval_samples_per_second': 1091.995, 'eval_steps_per_second': 34.143, 'epoch': 12.0}


100%|██████████| 26256/26256 [16:17<00:00, 26.87it/s]

{'train_runtime': 977.0725, 'train_samples_per_second': 859.711, 'train_steps_per_second': 26.872, 'train_loss': 4.382027002458962, 'epoch': 12.0}


TrainOutput(global_step=26256, training_loss=4.382027002458962, metrics={'train_runtime': 977.0725, 'train_samples_per_second': 859.711, 'train_steps_per_second': 26.872, 'train_loss': 4.382027002458962, 'epoch': 12.0})

In [17]:
trainer.evaluate(test_ds)

100%|██████████| 469/469 [00:13<00:00, 35.07it/s]


{'eval_loss': 4.383317470550537,
 'eval_top_one': 0.04919999837875366,
 'eval_top_five': 0.16333332657814026,
 'eval_mrr': 0.1257762461900711,
 'eval_top_ten': 0.2640666663646698,
 'eval_runtime': 13.5898,
 'eval_samples_per_second': 1103.771,
 'eval_steps_per_second': 34.511,
 'epoch': 12.0}

### Full Fine-Tuning

In [18]:
import os

In [20]:
for lr in [5e-5/100, 5e-5/300, 5e-5/1000]:
    os.makedirs(gpt2_dir/f"LPFT_100_{lr}", exist_ok=True)
    
    config = AutoConfig.from_pretrained("C:\\Users\\bunna\\Desktop\\classifier_100\\checkpoint-26256")
    config.num_labels = len(label2id)
    config.pad_token_id = tokenizer.pad_token_id
    model = AutoModelForSequenceClassification.from_pretrained("C:\\Users\\bunna\\Desktop\\classifier_100\\checkpoint-26256", config=config)

    optimizer = AdamW(
        model.parameters(),
        lr=lr,
    )

    num_training_steps = len(train_ds) // training_args.per_device_train_batch_size * training_args.num_train_epochs
    scheduler = get_scheduler(
        "cosine",
        optimizer,
        num_warmup_steps=0,  # You can adjust the warmup steps if needed
        num_training_steps=num_training_steps
    )

    training_args = TrainingArguments(
        output_dir=gpt2_dir/f"LPFT_100_{lr}",
        learning_rate=lr,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=5,
        weight_decay=0.01,
        logging_strategy="epoch",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        push_to_hub=False,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_ds,
        eval_dataset=val_ds,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
        optimizers=(optimizer, scheduler),
    )

    # Train model
    trainer.train()
    metrics = trainer.evaluate(test_ds)
    print(metrics)

c:\Users\bunna\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 20%|██        | 2188/10940 [02:57<11:44, 12.41it/s]

{'loss': 4.107, 'learning_rate': 4.914737092239488e-07, 'epoch': 1.0}


                                                    
 20%|██        | 2188/10940 [03:10<11:44, 12.41it/s]

{'eval_loss': 4.096588134765625, 'eval_top_one': 0.0860000029206276, 'eval_top_five': 0.24826666712760925, 'eval_mrr': 0.18203990161418915, 'eval_top_ten': 0.37786665558815, 'eval_runtime': 13.2238, 'eval_samples_per_second': 1134.315, 'eval_steps_per_second': 35.466, 'epoch': 1.0}


 40%|████      | 4376/10940 [06:09<08:18, 13.15it/s]  

...

{'eval_loss': 4.25885009765625, 'eval_top_one': 0.07046666741371155, 'eval_top_five': 0.2029999941587448, 'eval_mrr': 0.15506814420223236, 'eval_top_ten': 0.31966665387153625, 'eval_runtime': 13.5864, 'eval_samples_per_second': 1104.042, 'eval_steps_per_second': 34.52, 'epoch': 5.0}


100%|██████████| 10940/10940 [16:11<00:00, 11.26it/s]


{'train_runtime': 971.6554, 'train_samples_per_second': 360.21, 'train_steps_per_second': 11.259, 'train_loss': 4.198042768795704, 'epoch': 5.0}


100%|██████████| 469/469 [00:13<00:00, 34.83it/s]

{'eval_loss': 4.272396087646484, 'eval_top_one': 0.06599999964237213, 'eval_top_five': 0.20000000298023224, 'eval_mrr': 0.15055006742477417, 'eval_top_ten': 0.3124000132083893, 'eval_runtime': 13.5258, 'eval_samples_per_second': 1108.989, 'eval_steps_per_second': 34.674, 'epoch': 5.0}
